<h1> <b> Michelin restaurants in Italy Web scraping  </b> <img src="https://styles.redditmedia.com/t5_2s8x6/styles/communityIcon_ftfh5okvyxj01.png" width=120 style="vertical-align: middle"> </h1>

In today's world, people are more eager than ever to discover culinary experiences that are both unique and unforgettable. The culinary arts have evolved from a necessity to a refined skill, with passionate chefs pushing the boundaries of creativity and flavor. For food lovers, travelers, and students of gastronomy, resources like the Michelin Guide are invaluable. This platform, accessible via the <a href="https://guide.michelin.com/en/it/restaurants" > Michelin Guide website</a>, provides information, reviews, and ratings for restaurants throughout Italy and beyond, recognized for their quality, innovation, and exceptional culinary experiences.

Our team is dedicated to creating a search engine tailored for food lovers, helping users discover and rank Michelin-starred restaurants across Italy based on their unique preferences. Your mission: to create an efficient and intuitive tool to explore the best culinary experiences in Italy.

<h1> <b> Import Libraries </b> <img src="https://preview.redd.it/snoovatar/avatars/nftv2_bmZ0X2VpcDE1NToxMzdfZWI5NTlhNzE1ZGZmZmU2ZjgyZjQ2MDU1MzM5ODJjNDg1OWNiMTRmZV8yMTQ1NzYzNg_rare_46f1cdb1-634f-4c1d-8344-2be06c7880d4-headshot.png?width=256&height=256&crop=smart&auto=webp&s=400ead9440c7a9f06ca4c44953f24c5b765c4aac" width=150 style="vertical-align: middle"> </h1>

In [4]:
pip install beautifulsoup4 requests pandas tqdm nltk

Note: you may need to restart the kernel to use updated packages.


In [1]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import re
import time
from tqdm import tqdm
import math
import pickle
import heapq
import nltk

<h1> <b> 1.0.0 Data Collection </b> <img src="https://styles.redditmedia.com/t5_88fouv/styles/profileIcon_snoo-nftv2_bmZ0X2VpcDE1NToxMzdfZWI5NTlhNzE1ZGZmZmU2ZjgyZjQ2MDU1MzM5ODJjNDg1OWNiMTRmZV84MTg4MjQz_rare_177d88d7-3b80-4acf-b709-ba3f58c39a52-headshot.png" width=150 style="vertical-align: middle"> </h1> 

In [2]:
# Sends GET request to Michelin Guide website
response = requests.get("https://guide.michelin.com/en/it/restaurants")

In [3]:
soup = BeautifulSoup(response.content, features="lxml")

# Display all first 1000 characters of html
soup.prettify()[:1000]

'<!DOCTYPE html>\n<html class="full-screen-mobile" dir="" lang="en-US">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="width=device-width, initial-scale=1.0, user-scalable=0" name="viewport"/>\n  <meta content="" name="author"/>\n  <meta content="#fff" name="theme-color"/>\n  <meta content="MICHELIN Guide" property="og:site_name"/>\n  <meta content="Italy MICHELIN Restaurants – The MICHELIN Guide" itemprop="name"/>\n  <meta content="Italy MICHELIN Restaurants – The MICHELIN Guide" property="og:title"/>\n  <meta content="index, follow" name="robots"/>\n  <meta content="Article" property="og:type"/>\n  <meta content="https://guide.michelin.com/en/it/restaurants" property="og:url"/>\n  <meta content="Starred restaurants, Bib Gourmand and all the Restaurants of The MICHELIN Guide Italy. MICHELIN inspector reviews and insights" name="description"/>\n  <meta content="Starred restaurants, Bib Gourmand and all the Restaurants of The MICHELIN Guide Italy. MICHELIN inspector reviews and i

## 1.1. Get the list of Michelin restaurants

#### Our task is to collect the **URL** associated with each restaurant

In [4]:
# Define the base URLs for the Michelin guide page

full_base_url = "https://guide.michelin.com"  # Full base url for constructing restaurant links
base_url = "https://guide.michelin.com/en/it/restaurants"  # Base url for restaurant listings

# Initialize an empty list to store restaurant links
restaurant_links = []

# Loop through pages 1 to 100 
for page_num in tqdm(range(1, 101), desc="Reading pages", unit="page", colour="blue"):
    
    url = f"{base_url}/page/{page_num}"
    
    # Send a get request to fetch the content of the page
    response = requests.get(url)
    
    # Parse the page content using BeautifulSoup with the lxml parser
    soup = BeautifulSoup(response.content, features="lxml")
    
    links = soup.find_all('a', class_="link") # Finds all anchor tags with class 'link'
    
    # Extract and filter restaurant URLs from the 'href' attribute
    for link in links:
        href = link.get('href')
        if "/restaurant/" in href:  
            restaurant_links.append(full_base_url + href)
    
    # Pause for 2 seconds to avoid overwhelming the server
    time.sleep(2)

Reading pages: 100%|██████████| 100/100 [04:34<00:00,  2.74s/page]


In [46]:
#Let's check the number of restaurant

print(f"The number of restaurant links are : {len(restaurant_links)}")

The number of restaurant links are : 1981


In [6]:
#Visualize the first five restaurant link:

print(*restaurant_links[:5],sep="\n")   # * is used to unpack the elements of list 

https://guide.michelin.com/en/campania/gragnano/restaurant/o-me-o-il-mare
https://guide.michelin.com/en/abruzzo/popoli_1845563/restaurant/donevandro
https://guide.michelin.com/en/piemonte/alba/restaurant/ape-vino-e-cucina
https://guide.michelin.com/en/campania/sorrento/restaurant/da-bob-cook-fish
https://guide.michelin.com/en/basilicata/matera/restaurant/da-mo


## 1.2. Crawl Michelin restaurant pages
- ### Download the HTML corresponding to each of the collected URLs.
- ### After collecting each page, immediately save its HTML in a file. 
- ### Organize the downloaded HTML pages into folders. Each folder will contain the HTML of the restaurants from page 1, page 2, ... of the Michelin restaurant list.

In [7]:
# Define the base folder path for saving all restaurant HTML files for each page
base_save_path  = "Michelin Restaurants"

# Create the folder if it doesn’t already exist
os.makedirs(base_save_path , exist_ok=True)

In [8]:
for i in range(0, len(restaurant_links), 20):
    # Calculate the folder number based on the current range index
    folder_number = (i // 20) + 1
    
    # Create the folder to save the HTML files for this batch of urls
    folder_name = os.path.join(base_save_path, f"page {folder_number}")
    os.makedirs(folder_name, exist_ok=True)
    
    # Extract the group of URLs to download for this folder
    links_to_download = restaurant_links[i:i + 20]
    
    # Download each URL and save the HTML content
    for idx, url in enumerate(links_to_download):
        # Generate a filename for each restaurant's HTML file
        filename = os.path.join(folder_name, f"restaurant{i + idx + 1}.html")
        
        # Send a GET request to fetch the content of the restaurant url
        response = requests.get(url)
        
        # Save the HTML content of the restaurant into the file
        with open(filename, "w", encoding="utf-8") as file:
            file.write(response.text)


## 1.3 Parse downloaded pages

### At this point, we have all the HTML documents about the restaurant of interest, and we can start to extract specific information. The list of the information we desire for each restaurant and their format is :

- #### **Restaurant Name** (to save as `restaurantName`): string;
- #### **Address** (to save as `address`): string;
- #### **City** (to save as `city`): string;
- #### **Postal Code** (to save as `postalCode`): string;
- #### **Country** (to save as `country`): string;
- #### **Price Range**** (to save as `priceRange`): string;
- #### **Cuisine Type** (to save as `cuisineType`): string;
- #### **Description** (to save as `description`): string;
- #### **Facilities and Services** (to save as `facilitiesServices`): list of strings;
- #### **Accepted Credit Cards** (to save as `creditCards`): list of strings;
- #### **Phone Number** (to save as `phoneNumber`): string;
- #### **URL** to the **Restaurant Page** (to save as `website`): string.

In [9]:
def extract_restaurant_data(file_path):
    # Load and parse the HTML file
    with open(file_path, "r", encoding="utf-8") as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, "html.parser")

    # Initialize the restaurant_info dictionary with default values
    restaurant_info = {
        'restaurantName': None,
        'address': None,
        'city': None,
        'postalCode': None,
        'country': None,
        'priceRange': None,
        'cuisineType': None,
        'description': None,
        'facilitiesServices': [],
        'creditCards': [],
        'phoneNumber': None,
        'website': None
    }

    # Extract Restaurant Name
    title_tag = soup.find('h1', class_='data-sheet__title')
    if title_tag:
        restaurant_info['restaurantName'] = title_tag.get_text(strip=True)

    # Extract Address Components
    address_text_tag = soup.find("div", class_="data-sheet__block--text")
    address_text = address_text_tag.get_text(strip=True)
    address_text=re.sub(r'\bloc\.\s*[^,]*,', '', address_text) # Remove location (loc) and the next part up to the comma
    if address_text:
        address_parts = address_text.replace('\n', ' ').split(",")
        restaurant_info["address"] = address_parts[0] if len(address_parts) > 0 else None
        restaurant_info["city"] = address_parts[1] if len(address_parts) > 1 else None
        restaurant_info["postalCode"] = address_parts[2] if len(address_parts) > 2 else None
        restaurant_info["country"] = address_parts[3] if len(address_parts) > 3 else None

    # Extract Price Range and Cuisine Type
    info_divs = soup.find_all("div", class_="data-sheet__block--text")
    if len(info_divs) > 1:
        info_text = info_divs[1].text.strip()
        if '·' in info_text:
            info_parts = info_text.split('·')
            restaurant_info['priceRange'] = info_parts[0].strip()
            if len(info_parts) > 1:
                restaurant_info['cuisineType'] = info_parts[1].strip()

    # Extract Description
    description_tag = soup.select_one('.data-sheet__description')
    if description_tag:
        restaurant_info['description'] = description_tag.get_text(strip=True)

    # Extract Facilities and Services
    facilities_services = [service.get_text(strip=True) for service in soup.select('.restaurant-details__services li')]
    if facilities_services:
        restaurant_info['facilitiesServices'] = facilities_services

    # Extract Accepted Credit Cards (Modify this part to return a list of credit cards)
    cards=[]
    credit_cards = soup.find_all("img", class_="lazy", height="32")  # Modify as per the HTML structure
    for img in credit_cards:
        src = img.get('src', '') or img.get('data-src', '')
        if src:# Check if 'src' is not empty
            text=src.split("/")[-1]
            cards.append(text.split("-")[0].capitalize())
    restaurant_info["creditCards"]=cards

    # Extract Phone Number
    phone_span = soup.select_one('.collapse__block-item span')
    if phone_span:
        restaurant_info['phoneNumber'] = phone_span.get_text(strip=True)

    # Extract Website URL
    website_link = soup.select_one('.collapse__block-item.link-item a')
    if website_link:
        restaurant_info['website'] = website_link['href']

    return restaurant_info

#### Let's define a sorting method to apply to the `sorted` function to correctly sort the folders of each page

In [10]:
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

#### We define a `parse_all_folders` function to create a dataset composed of the specific information of each html for each restaurant of each page

In [11]:
def parse_all_folders(base_folder):

    """ Parse all HTML files in a folder structure and aggregate into a DataFrame."""

    data = []  # List to store all parsed data
    
    # Traverse through each folder and file in the base_folder
    for folder_name in tqdm(sorted(os.listdir(base_folder),key=natural_sort_key),desc="Read folder of each page",unit="page"):
        folder_path = os.path.join(base_folder, folder_name)
        
        # Check if the path is a directory
        if os.path.isdir(folder_path):
            for filename in sorted(os.listdir(folder_path),key=natural_sort_key):
                # Only process HTML files
                if filename.endswith(".html"):
                    file_path = os.path.join(folder_path, filename)
                    # Parse the file and add to data list
                    restaurant_data = extract_restaurant_data(file_path)
                    data.append(restaurant_data)
    
    # Create DataFrame from collected data
    df = pd.DataFrame(data)
    return df

In [108]:
# Usage :
base_save_path  = "Michelin Restaurants"
data = parse_all_folders(base_save_path)

# Save the DataFrame to a CSV file
data.to_csv("all_restaurants_data.csv", index=False)

Read folder of each page: 100%|██████████| 100/100 [03:07<00:00,  1.87s/page]


In [ ]:
data=pd.read_csv("all_restaurants_data.csv",index=False)

In [109]:
data.head()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,O Me O Il Mare,Via Roma 45/47,Gragnano,80054,Italy,€€€€,"Italian Contemporary, Modern Cuisine","Known around the world as the town of pasta, Gragnano is still home to small-scale pasta-makers renowned for their top-quality pasta. To reinforce this fact, this restaurant is housed in a building (dating back to 1695) once used as pasta factory. Now boasting a modern, spacious dining room with a vaulted ceiling and open-view kitchen, the restaurant serves three tasting menus, all with strong links to the region but also featuring more creative touches. The region also takes pride of place on the wine list – it’s worth asking the talented and experienced sommelier for her recommendations.","[Air conditioning, Interesting wine list, Wheelchair access]","[Amex, Dinersclub, Mastercard, Visa]",+39 081 620 0550,http://omeoilmare.com
1,Donevandro,via Garibaldi 2,Popoli,65026,Italy,€€,"Contemporary, Seasonal Cuisine","Up until a few years ago, the owner-chef at this restaurant was working as a painter – a fact that is evident from the artistic touch in his cuisine. His recipes are modern and personalised, with careful attention naturally paid to harmonious presentation, while the flavour of his dishes is brought out by ingredients that are skilfully chosen from the Abruzzo inland area. In 2024, the restaurant moved to new, centrally located premises which have an intimate feel and are elegant and minimalist in style.",[Air conditioning],"[Mastercard, Visa]",+39 388 887 6858,http://www.donevandroristorante.it
2,Ape Vino e Cucina,Piazza Risorgimento 3,Alba,12051,Italy,€€,"Piedmontese, Contemporary","This attractive restaurant in the heart of Alba, the gastronomic capital of the Langhe, is home to chef Damiano Nigro who has previous experience in Michelin-starred restaurants in the area. The restaurant serves aperitif snacks, tapas and top-quality cocktails alongside its delicious, generously portioned meals which, as you might expect in Piedmont, have a traditional flavour. The pasta, bread, desserts and ice-cream are all homemade, while on our last visit in the summer we particularly enjoyed the maccheroncini with scampi and peppers, and the exquisite Fassona beef with a red-wine sauce and seasonal vegetables. Booked ahead is recommended.","[Air conditioning, Terrace, Wheelchair access]","[Amex, Dinersclub, Maestrocard, Mastercard, Visa]",+39 0173 363453,https://www.apewinebar.it/alba/
3,Da Bob Cook Fish,largo Parsano vecchio 16,Sorrento,80067,Italy,€€,Seafood,"Working in partnership with the nearby fishmongers which supplies it with fresh fish and seafood, this restaurant occupies a building once used for storing oranges that dates back 400 years. Although there is a menu, given the restaurant’s close relationship with the fishmongers we recommend asking the serving staff about the catch of the day and taking a look in the glass cabinet where the fish are displayed. Dishes are classic and traditional, allowing the full flavour of the fresh fish to shine through.","[Air conditioning, Terrace]","[Amex, Dinersclub, Mastercard, Visa]",+39 081 1778 3873,https://www.dabobcookfish.com/
4,DA_MÓ,Via Bruno Buozzi 20,Matera,75100,Italy,€€,"Regional Cuisine, Contemporary","This new, restored restaurant in the upper part of the Sassi (yet still near the centre of Matera and its car parks) has a warm friendly ambience and boasts a small, charming outdoor space. Here, an entire family (father in the kitchen, mother and daughter front of house), who moved to Matera from Venosa (home of the Latin poet Horatio), serves regional cuisine with a contemporary twist and no little skill and professionalism. Various tasting menus are available, as well as an à la carte. An interesting wine list completes the picture.","[Air conditioning, Terrace]","[Amex, Dinersclub, Mastercard, Visa]",+39 0835 686548,https://www.damoristorante.i

In [110]:
data.tail()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
1976,Umami,Via Ugo Secondo Partigiano 1,Badalucco,18010,Italy,€€,Modern Cuisine,"A young chef with experience in renowned restaurants has taken over the old kindergarten in this village and transformed it into a delightful restaurant. Here, he serves contemporary cuisine, often influenced by local traditions and showcased on tasting menus from which individual dishes can be ordered à la carte style. In summer, there’s an attractive outdoor space practically on the banks of the Argentina river, which gives its name to the valley in which this village is situated.","[Terrace, Wheelchair access]","[Amex, Mastercard, Visa]",+39 331 338 6005,https://www.umamirestaurant.it/
1977,Visione Restaurant and Living,Strada Nicolini Basso 34,Barbaresco,12050,Italy,€€€,"Contemporary, Piedmontese","At this restaurant, new, young and enthusiastic managers work alongside a chef with Asian origins (who grew up in Italy) to offer a wide selection of dishes, many with an exotic flavour. Choose from an à la carte or two tasting menus, one of which has its roots in the regional traditions that are always held in high regard in Piedmont. Situated in the Casa Nicolini hotel in a quiet, panoramic location, the restaurant boasts fine views of the hills through its dining room windows and from its terrace, where a bar and bistro with a more concise and reasonably priced menu are set up in the summer months. The bistro also has its own small, dedicated dining room.","[Air conditioning, Car park]","[Amex, Maestrocard, Mastercard, Visa]",+39 328 134 0218,https://www.ristorantevisione.it
1978,Ristorante de LEN,Via Cesare Battisti 66,Cortina d'Ampezzo,32043,Italy,€€,Regional Cuisine,"Just a stone’s throw from the central and very busy Corso Italia, this restaurant has an elegant Alpine feel. Here, subtle lighting, soft background music and larch wood cladding on the walls create an ambience with an attractive blend of the modern and traditional. The cuisine takes inspiration from the local region, with specialities interpreted with a contemporary twist, including a focus on vegetable- and cheese-based dishes such as the excellent “fonduta di Saporito d'alpeggio” (fondue).",[Wheelchair access],"[Amex, Dinersclub, Mastercard, Visa]",+39 0436 4246,https://hoteldelen.it
1979,AceroRosso,Via Ruvignan 1,Vodo di Cadore,32040,Italy,€€,Regional Cuisine,"This secluded mountain chalet immersed in verdant woodland boasts a welcoming dining room with a decidedly Alpine feel thanks to the profusion of wood and a warming open fire. The cuisine is also regional in style, featuring local recipes and ingredients albeit with a contemporary touch. Popular dishes include the rack of lamb cooked at low temperature and served with creamed sweetcorn, leek fondant and its own jus.","[Car park, Terrace, Wheelchair access]","[Amex, Mastercard, Visa]",+39 0435 489653,https://www.acerorossodolomiti.it
1980,Café Les Paillotes,piazza Le Laudi 2,Pescara,65129,Italy,€€€,"Modern Cuisine, Seafood","This old acquaintance of the Michelin Guide now has a new chef at the helm! Although the decor and furnishings remain the same with a slightly old-fashioned feel, the cuisine is modern yet with no unnecessary frills and has a fresh and generous flavour. Highlights on the menu include the new raw fish options, which have an entire tasting menu to themselves. The well-stocked wine cellar is home to around 500 labels which offer good value for money.","[Air conditioning, Interesting wine list, Restaurant offering vegetarian menus, Terrace, Wheelchair access]","[Amex, Dinersclub, Mastercard, Visa]",+39 085 61809,https://www.lespaillotes.it/


In [111]:
data.shape

(1981, 12)

<h1> <b> 2.0.0 Search Engine and preprocessing the Text </b> <img src= "https://cdn-icons-png.flaticon.com/256/2857/2857376.png" width=100 style="vertical-align: middle"> </h1>

In [112]:
# Mapping price_range column for more readability

price_range_labels = {'€': 'Economic',  '€€': 'Affordable','€€€': 'Expensive','€€€€': 'Luxury'}

data['priceRange'] = data['priceRange'].map(price_range_labels)

In [113]:
data.head()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,O Me O Il Mare,Via Roma 45/47,Gragnano,80054,Italy,Luxury,"Italian Contemporary, Modern Cuisine","Known around the world as the town of pasta, Gragnano is still home to small-scale pasta-makers renowned for their top-quality pasta. To reinforce this fact, this restaurant is housed in a building (dating back to 1695) once used as pasta factory. Now boasting a modern, spacious dining room with a vaulted ceiling and open-view kitchen, the restaurant serves three tasting menus, all with strong links to the region but also featuring more creative touches. The region also takes pride of place on the wine list – it’s worth asking the talented and experienced sommelier for her recommendations.","[Air conditioning, Interesting wine list, Wheelchair access]","[Amex, Dinersclub, Mastercard, Visa]",+39 081 620 0550,http://omeoilmare.com
1,Donevandro,via Garibaldi 2,Popoli,65026,Italy,Affordable,"Contemporary, Seasonal Cuisine","Up until a few years ago, the owner-chef at this restaurant was working as a painter – a fact that is evident from the artistic touch in his cuisine. His recipes are modern and personalised, with careful attention naturally paid to harmonious presentation, while the flavour of his dishes is brought out by ingredients that are skilfully chosen from the Abruzzo inland area. In 2024, the restaurant moved to new, centrally located premises which have an intimate feel and are elegant and minimalist in style.",[Air conditioning],"[Mastercard, Visa]",+39 388 887 6858,http://www.donevandroristorante.it
2,Ape Vino e Cucina,Piazza Risorgimento 3,Alba,12051,Italy,Affordable,"Piedmontese, Contemporary","This attractive restaurant in the heart of Alba, the gastronomic capital of the Langhe, is home to chef Damiano Nigro who has previous experience in Michelin-starred restaurants in the area. The restaurant serves aperitif snacks, tapas and top-quality cocktails alongside its delicious, generously portioned meals which, as you might expect in Piedmont, have a traditional flavour. The pasta, bread, desserts and ice-cream are all homemade, while on our last visit in the summer we particularly enjoyed the maccheroncini with scampi and peppers, and the exquisite Fassona beef with a red-wine sauce and seasonal vegetables. Booked ahead is recommended.","[Air conditioning, Terrace, Wheelchair access]","[Amex, Dinersclub, Maestrocard, Mastercard, Visa]",+39 0173 363453,https://www.apewinebar.it/alba/
3,Da Bob Cook Fish,largo Parsano vecchio 16,Sorrento,80067,Italy,Affordable,Seafood,"Working in partnership with the nearby fishmongers which supplies it with fresh fish and seafood, this restaurant occupies a building once used for storing oranges that dates back 400 years. Although there is a menu, given the restaurant’s close relationship with the fishmongers we recommend asking the serving staff about the catch of the day and taking a look in the glass cabinet where the fish are displayed. Dishes are classic and traditional, allowing the full flavour of the fresh fish to shine through.","[Air conditioning, Terrace]","[Amex, Dinersclub, Mastercard, Visa]",+39 081 1778 3873,https://www.dabobcookfish.com/
4,DA_MÓ,Via Bruno Buozzi 20,Matera,75100,Italy,Affordable,"Regional Cuisine, Contemporary","This new, restored restaurant in the upper part of the Sassi (yet still near the centre of Matera and its car parks) has a warm friendly ambience and boasts a small, charming outdoor space. Here, an entire family (father in the kitchen, mother and daughter front of house), who moved to Matera from Venosa (home of the Latin poet Horatio), serves regional cuisine with a contemporary twist and no little skill and professionalism. Various tasting menus are available, as well as an à la carte. An interesting wine list completes the picture.","[Air conditioning, Terrace]","[Amex, Dinersclub, Mastercard, Visa]",+39 0835 6

In [114]:
data["description"].iloc[0]

'Known around the world as the town of pasta, Gragnano is still home to small-scale pasta-makers renowned for their top-quality pasta. To reinforce this fact, this restaurant is housed in a building (dating back to 1695) once used as pasta factory. Now boasting a modern, spacious dining room with a vaulted ceiling and open-view kitchen, the restaurant serves three tasting menus, all with strong links to the region but also featuring more creative touches. The region also takes pride of place on the wine list – it’s worth asking the talented and experienced sommelier for her recommendations.'

In [115]:
# visualize all token of the first description of dataset
data["description"].iloc[0].split()

['Known',
 'around',
 'the',
 'world',
 'as',
 'the',
 'town',
 'of',
 'pasta,',
 'Gragnano',
 'is',
 'still',
 'home',
 'to',
 'small-scale',
 'pasta-makers',
 'renowned',
 'for',
 'their',
 'top-quality',
 'pasta.',
 'To',
 'reinforce',
 'this',
 'fact,',
 'this',
 'restaurant',
 'is',
 'housed',
 'in',
 'a',
 'building',
 '(dating',
 'back',
 'to',
 '1695)',
 'once',
 'used',
 'as',
 'pasta',
 'factory.',
 'Now',
 'boasting',
 'a',
 'modern,',
 'spacious',
 'dining',
 'room',
 'with',
 'a',
 'vaulted',
 'ceiling',
 'and',
 'open-view',
 'kitchen,',
 'the',
 'restaurant',
 'serves',
 'three',
 'tasting',
 'menus,',
 'all',
 'with',
 'strong',
 'links',
 'to',
 'the',
 'region',
 'but',
 'also',
 'featuring',
 'more',
 'creative',
 'touches.',
 'The',
 'region',
 'also',
 'takes',
 'pride',
 'of',
 'place',
 'on',
 'the',
 'wine',
 'list',
 '–',
 'it’s',
 'worth',
 'asking',
 'the',
 'talented',
 'and',
 'experienced',
 'sommelier',
 'for',
 'her',
 'recommendations.']

### Before building the search engine, you must clean and prepare the text in each restaurant’s description. We will:

- #### 1. Remove stopwords ;
- #### 2. Remove punctuation ;
- #### 3. Apply stemming ;
- #### 4. Perform any other necessary cleaning to improve search accuracy.

### For the Preprocess operation of all description we use the `nltk` library

In [116]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Initialize stop words and stemmer
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")


def text_cleaning(text):
    
    text=text.lower()

    # Remove special characters and symbols (non-alphanumeric characters)
    text = re.sub(r'[^\w\s]', '', text)

    # Remove extra spaces and numbers
    text = re.sub(r'\s+', '  ', text)
    text = re.sub(r'\d+', ' ', text)

    # Tokenize the text (split it into individual words)
    words = word_tokenize(text)

    # Apply filters, remove stopwords, exclude verbs, and apply stemmmer
    processed_words = [
        stemmer.stem(word)  
        for word in words
        if word not in stop_words and nltk.pos_tag([word])[0][1] not in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]  ] # remove all stopwords, verbs from the description and apply the stemmer


    processed_text = ' '.join(processed_words).strip()

    return processed_text


# Apply the preprocessing function 
tqdm.pandas(desc="Processing descriptions")  # Setup tqdm progress bar
data['description2'] = data['description'].progress_apply(text_cleaning)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flavi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flavi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\flavi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\flavi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Processing descriptions: 100%|██████████| 1981/1981 [00:34<00:00, 57.00it/s]


We have done preprocessing on the restaurant descriptions using the nltk library, following these steps:

*  Removal of stopwords: We imported English stopwords and kept only words that are not among them ;
*  Removal of punctuation ;
*  Application of stemming: We used the PorterStemmer to reduce words to their root ;
*  Other filters: We converted words to lowercase and accepted only alphabetical words.

At the end these operations were applied to the `description` column of the dataset, and the result was saved in a new column `description2` , ready for use in a search engine.

In [118]:
# Set display options to show full text in each cell
pd.set_option('display.max_colwidth', None)

# Display the first few rows of 'description2'
print(data[['description2']].iloc[0])

description2    around world town pasta gragnano still home smallscal pastamak topqual pasta reinforc fact restaur build back pasta factori boast modern spacious room ceil openview kitchen restaur serv three menus strong link region also creativ touch region also pride place wine list worth experienc sommeli recommend
Name: 0, dtype: object


In [119]:
#create a set of all noduplicated words in the variable 'description2'
vocabulary=set()
for text in data['description2']:
    vocabulary.update(text.split())
print(f"number of words in vocabulary : {len(vocabulary)} words!")

#create a dictionary_word with all noduplicated words as keys and the associated term_id as values
dictionary_word={}
for word,word_id in enumerate(vocabulary,start=0):
    dictionary_word[word]=word_id

number of words in vocabulary : 7056 words!


In [120]:
#save dictionary_word in a csv file named "vocabulary.csv"
dictionary_word_df=pd.DataFrame(dictionary_word.items(),columns=['word_id','word'])
dictionary_word_df.to_csv('vocabulary.csv',index=False)

In [121]:
#show first rows of vocabulary.csv
vocabulary=pd.read_csv('vocabulary.csv')
vocabulary.head()

,word_id,word
0,0,anagram
1,1,enbogonè
2,2,white
3,3,luca
4,4,cheek


## 2.1 Conjunctive Query
#### This first version of the search engine narrows the search to the description field of each restaurant. Only restaurants whose descriptions contain all the query words will be returned.

### 2.1.1 Create your Index!

* ##### Vocabulary File: Create a file called vocabulary.csv that maps each word to a
unique integer (term_id);
* ##### Inverted Index: Build a dictionary_word mapping each term_id to a list of document IDs where that term appears.
{
  "term_id_1": [document_1, document_2, document_4],
  "term_id_2": [document_1, document_3, document_5],
  ...
}

#### **Each document_i represents a unique restaurant.**

#### We've created a vocabulary of words from the description2 column. We assigned a unique term_id to each word and saved it to vocabulary.csv. This step is necessary for the construction of the inverted index.

In [122]:
# Initialize the inverted index and dictionary_word for restaurant names
inverted_index = {}
dictionary_word_restaurant = {}

# Create dictionary_word for term to term_id mapping
# Assuming 'dictionary_word' will store words as keys and their corresponding term_ids as values
# Populate the dictionary_word for each unique restaurant name
for document_id, restaurant_name in enumerate(data['restaurantName'], start=0):
    dictionary_word_restaurant[restaurant_name] = document_id  # Map restaurant name to document id

# Create vocabulary from the 'description2' column (unique words)
vocabulary = set()  # Set to store unique words
for desc in data['description2']:
    # Tokenize the description and add unique words to the vocabulary
    vocabulary.update(desc.split())

# Create dictionary_word for terms (words) to term_ids mapping
dictionary_word = {word: term_id for term_id, word in enumerate(vocabulary, start=0)}

# Initialize the inverted index from the 'description2' column
for i in range(len(data)):
    # Extract document_id from dictionary_word_restaurant using the restaurant name
    document_id = dictionary_word_restaurant[data['restaurantName'].iloc[i]]
    words = data['description2'].iloc[i].split()  # Split description2 into words
    
    # For each word in the description2
    for word in words:
        if word in dictionary_word:  # Ensure the word exists in the dictionary_word
            term_id = dictionary_word[word]  # Extract term_id from dictionary_word
            
            # Initialize the inverted index entry for this term_id if not present
            if term_id not in inverted_index:
                inverted_index[term_id] = []
            
            # Add the document_id to the inverted index for this term_id if it's not already present
            if document_id not in inverted_index[term_id]:
                inverted_index[term_id].append(document_id)

# Print the first word entries of the vocabulary (word, term_id pairs)
print("Vocabulary:", list(dictionary_word.items())[:1])

# Show the first word entries of the inverted index for inspection
print("Inverted Index:", list(inverted_index.items())[:1])

# Check if the number of unique term_ids in vocabulary matches the term_ids in the inverted index
if len(vocabulary) == len(inverted_index):
    print("The vocabulary and inverted index match.")
else:
    print("There is a mismatch between vocabulary and inverted index.")


Vocabulary: [('anagram', 0)]
Inverted Index: [(2145, [0, 10, 14, 25, 29, 32, 50, 65, 76, 97, 101, 109, 135, 159, 180, 189, 221, 293, 336, 353, 358, 375, 381, 435, 438, 444, 505, 516, 523, 567, 610, 615, 648, 663, 696, 704, 730, 732, 734, 754, 758, 760, 764, 809, 847, 853, 872, 889, 905, 919, 936, 937, 1012, 1043, 1046, 1088, 1109, 1175, 1189, 1223, 1247, 1255, 1279, 1280, 1288, 1315, 1345, 1352, 1353, 1380, 1425, 1435, 1454, 1573, 1578, 1591, 1665, 1703, 1708, 1757, 1781, 1786, 1848, 1909, 1923, 1929, 1931, 1934, 1941, 1949, 1953, 1980])]
The vocabulary and inverted index match.


**We've created :**
* A dictionary_word for restaurants, dictionary_word_restaurant, where each key is a unique restaurant name from the restaurantName column, and each value is a unique document_id ;
* A dictionary_word inverted_index, which contains term_id values as keys, each linked to a list of document_id values where the words appear.

Then we checked that inverted_index contains all the words from description2 by comparing the length of vocabulary with the number of keys in inverted_index. This confirms that all words are included in the inverted index

In [123]:
# use pickle library

# saved inverted index in "inverted_index.pkl"
with open("inverted_index.pkl","wb") as f:
    pickle.dump(inverted_index,f)
# show inverted index
with open("inverted_index.pkl","rb") as f:
    loaded_inverted_index=pickle.load(f)

## 2.1.2 Execute the Query

#### **When the user inputs a query, for example, "modern seasonal cuisine", the search engine will:**

In [124]:
def execute_query(query):
    
    # Preprocess the query: remove punctuation, convert to lowercase
    query =  query.lower().split()

    # Create an empty list to store the word IDs corresponding to the query terms
    query_word_ids = []

    # Iterate over each word in the preprocessed query
    for word in query:
        # If the word exists in the dictionary (which maps words to IDs)
        if word in dictionary_word:
            # Append the corresponding word ID to the query_word_ids list
            query_word_ids.append(dictionary_word[word])

    # Check if we have any valid word IDs in the query
    if not query_word_ids:
        return "No matching words found in the dictionary."

    # Initialize the intersection list to store the document IDs that match the query
    intersection = []

    # If there are multiple query words, find the intersection of document IDs for all query terms
    if len(query_word_ids) >= 1:
        # Start with the set of document IDs corresponding to the first word in the query
        initial_set = set(inverted_index.get(query_word_ids[0], []))
        
        # For each remaining word ID in the query, find the intersection of document IDs
        for word_id in query_word_ids:
            current_set = set(inverted_index.get(word_id, []))
            intersection_set = initial_set & current_set  # Update the intersection
        
        # Convert the final intersection set to a list
        intersection = list(intersection_set)
    else:
        # If there is only one query word, use its document IDs directly
        intersection = list(set(inverted_index.get(query_word_ids[0], [])))

    # Check if the intersection is empty
    if not intersection:
        return "No documents match the query."

    # Initialize the list to store restaurant names that match the document IDs in the intersection
    restaurant_names = []

    # For each document ID in the intersection
    for doc_id in intersection:
        # Check if this document ID corresponds to any restaurant in the dictionary
        for name, rest_id in dictionary_word_restaurant.items():
            if rest_id == doc_id:
                # If the document ID matches, add the restaurant name to the list
                restaurant_names.append(name)

    # Check if any restaurants were found
    if not restaurant_names:
        return "No matching restaurants found."

    # Filter the original data to only include the restaurants in the restaurant_names list
    results = data[data["restaurantName"].isin(restaurant_names)]
    
    # Select only the relevant columns for the final results
    results = results[['restaurantName', 'address', 'description', 'website']]

    # Return the search results as a DataFrame
    return results




* Processed query words: the function cleans and converts all query words to ensure compatibility with the inverted index
*  Conjunctive Query: an intersection of all document_ids for the query words is performed, so that only restaurants containing all the words in the query are returned
*  Output: the result is a DataFrame with the columns {restaurantName, address, description, website}

We have implemented the conjunctive search engine, and the output provides results for the query 'modern seasonal cuisine'




In [125]:
query = input("Please enter a short description of the type of restaurant you're looking for:\n"
              "For example: 'modern seasonal cuisine with a terrace' or 'romantic Italian restaurant'.\n\n"
              "Your description: ")

results = execute_query(query)

#Show the results
results[:5]

Please enter a short description of the type of restaurant you're looking for:
For example: 'modern seasonal cuisine with a terrace' or 'romantic Italian restaurant'.

Your description:  modern seasonal cuisine


,restaurantName,address,description,website
0,O Me O Il Mare,Via Roma 45/47,"Known around the world as the town of pasta, Gragnano is still home to small-scale pasta-makers renowned for their top-quality pasta. To reinforce this fact, this restaurant is housed in a building (dating back to 1695) once used as pasta factory. Now boasting a modern, spacious dining room with a vaulted ceiling and open-view kitchen, the restaurant serves three tasting menus, all with strong links to the region but also featuring more creative touches. The region also takes pride of place on the wine list – it’s worth asking the talented and experienced sommelier for her recommendations.",http://omeoilmare.com
1,Donevandro,via Garibaldi 2,"Up until a few years ago, the owner-chef at this restaurant was working as a painter – a fact that is evident from the artistic touch in his cuisine. His recipes are modern and personalised, with careful attention naturally paid to harmonious presentation, while the flavour of his dishes is brought out by ingredients that are skilfully chosen from the Abruzzo inland area. In 2024, the restaurant moved to new, centrally located premises which have an intimate feel and are elegant and minimalist in style.",http://www.donevandroristorante.it
8,La Buca,corso Garibaldi 45,"Choose one of the tables on the outdoor summer terrace (right over the water!) at this restaurant in order to best appreciate its location overlooking the picturesque canal port and its period houses. The indoor dining room is also attractive, with its open-view kitchen and a modern and elegant feel, while the menu focuses almost exclusively on fish, most of which is sourced from local fishermen. Enjoy a wide selection of raw antipasti to start (some simple in their preparation, while others are more elaborate), then choose from an array of beautifully presented, modern dishes, perhaps accompanied by one of the many champagnes included on the wine list.",https://www.labucaristorante.com/
11,Il Ristorante Alain Ducasse Napoli,Via Cristoforo Colombo 45,"Alain Ducasse, one of the great names in contemporary fine dining, has arrived in Naples, opening this restaurant in the former premises of the prestigious Ristorante Il Comandante. Situated on the 9th floor of the Romeo hotel overlooking the port, the restaurant boasts fine views of Vesuvius and the Bay of Naples, especially at sunset when the colours are truly spectacular. Meanwhile, the modern dining room, decorated completely in black, is equally as stunning as its surroundings. Alessandro Lucassino, who was born in 1991 and has years of experience working with his mentor, adds a personal flavour to local recipes: by using short cooking times, he preserves the nutritious qualities and flavours of local fish and vegetables while remaining faithful to Ducasse’s “cuisine de la naturalité” philosophy.",https://theromeocollection.com/en/romeo-napoli/restaurants-bars/il-ristorante-alain-ducasse/
14,Etra,piazza De Ferrari 4,"Etra is an anagram of the Italian word “arte” – an apt name for this restaurant, which is housed in Palazzo Doria De Fornari on the beautiful and famous Piazza Ferrari. This palazzo is one of Genova’s magnificent “Rolli” palazzi (noble palaces once used to house famous visitors to the city). Despite the setting, the dining room is modern in style and adorned with elegant works of contemporary art. Here, chef Davide Cannavino consolidates his reputation with a concise menu of creative dishes: around a dozen meat and fish options inspired by the beauty that surrounds him and showcased on two tasting menus from which individual dishes can be chosen à la carte style.",https://www.etra.art/


## 2.2 Ranked Search Engine with TF-IDF and Cosine Similarity
### For the second search engine, given a query, retrieve the top-k restaurants ranked by relevance to the query.

### 2.2.1 Inverted Index with TF-IDF Scores
- #### tfIdf Scores: Calculate TF-IDF scores for each term in each restaurant’s description.
- #### Updated Inverted Index: Build a new inverted index where each entry is a term, and the value is a list of tuples containing document IDs and TF-IDF scores.

In [126]:
def calculate_tf(data, dictionary_word_restaurant):
    """Calculate term frequency (TF) for each document and track document appearances for each word."""
    tf = {}
    dict_word_doc_id = {}

    # Iterate over each document in the dataset
    for idx, row in data.iterrows():
        # Get the document ID from the 'restaurantName'
        document_id = dictionary_word_restaurant.get(row['restaurantName'])

        # Getting the list of words from the preprocessed description
        words = row['description2'].split()

        # Initialize a dictionary for TF scores for this document
        tf[document_id] = {}

        for word in words:
            # Increment the word count in the current document's dictionary
            tf[document_id][word] = tf[document_id].get(word, 0) + 1

            # Track the document ID for the current word
            if word not in dict_word_doc_id:
                dict_word_doc_id[word] = set()
            dict_word_doc_id[word].add(document_id)

        # Normalize TF values by dividing by the total number of words in the document
        num_words = len(words)
        for word in tf[document_id]:
            tf[document_id][word] /= num_words

    return tf, dict_word_doc_id



* ### Calculated TF-IDF scores: the calculate_tf_idf function computed the TF-IDF scores for each word in the descriptions.
* ### Created updated inverted index: this inverted index uses term_id as keys and stores tuples of (document_id, tfidf_score) as values.

This updated inverted index is now ready to be used in the Ranked Search Engine with TF-IDF and Cosine Similarity


In [127]:
def calculate_tf_idf(data, dictionary_word_restaurant):
    """Calculates TF-IDF for each document and creates the inverted index."""
    # Calculate TF and build the document index for each word
    tf, dict_word_doc_id = calculate_tf(data, dictionary_word_restaurant)
    N = len(data)  # Total number of documents
    
    tf_idf = {}  # Dictionary to store TF-IDF scores

    # Calculate IDF and TF-IDF
    for document_id, word_tf in tf.items():
        tf_idf[document_id] = {}
        for word, tf_score in word_tf.items():
            # Calculate IDF
            idf = math.log(N / len(dict_word_doc_id[word]))
            # Calculate TF-IDF as tf * idf
            tf_idf[document_id][word] = tf_score * idf

    return tf_idf, dict_word_doc_id


In [128]:
def create_inverted_index(tf_idf, dictionary_word):
    """Create an inverted index mapping terms to documents and their TF-IDF scores."""
    inverted_index = {}
    
    for document_id, word_tfidf in tf_idf.items():
        for word, tfidf_score in word_tfidf.items():
            # Get the ID for the term from the dictionary
            term_id = dictionary_word.get(word)
            
            if term_id is None:
                raise KeyError(f"Term '{word}' not found in dictionary_word.")
            
            # Add the document and the TF-IDF score to the term's list in the inverted index
            if term_id not in inverted_index:
                inverted_index[term_id] = []
            inverted_index[term_id].append((document_id, tfidf_score))
    
    return inverted_index

# Execute the TF-IDF calculation and create the inverted index
tf_idf, dict_word_doc_id = calculate_tf_idf(data, dictionary_word_restaurant)
inverted_index = create_inverted_index(tf_idf, dictionary_word)

## 2.2.2 Execute the Ranked Query

### Ranking Restaurants using Cosine Similarity

The process involves ranking restaurants based on a search query through the following steps:

#### 1. **Process the Query Terms**: The query terms are extracted and prepared.
#### 2. **Calculate TF-IDF Vectors**: Compute the TF-IDF vectors for both the query and each restaurant document.
#### 3. **Compute Cosine Similarity**: Compare the cosine similarity between the query and each restaurant’s TF-IDF vector.
#### 4. **Return Top-k Results**: Return the top-k restaurants or fewer if there are less than k matches with non-zero similarity.
#### 5. **Result Details**: Each result should include:
   - `Restaurant Name`
   - `Address`
   - `Description`
   - `Website`
   - `Similarity Score (between 0 and 1)`


### Now create restaurant_info_dict as a base to access restaurant details, like restaurant_name, address, description , website etc... This restaurant_info_dict will be used in the next steps

In [129]:
restaurant_info_dict={}

for i in range(len(data)):
    document_id=dictionary_word_restaurant[data['restaurantName'].iloc[i]]  #used dictionary_word_restaurant created previously to obtain `document_id`

    if document_id not in restaurant_info_dict:
        restaurant_info_dict[document_id]={
            "restaurantName": data['restaurantName'].iloc[i],
            "address": data['address'].iloc[i],
            "description": data['description'].iloc[i],
            "website": data['website'].iloc[i],
            "cuisineType": data['cuisineType'].iloc[i],
            "priceRange": data['priceRange'].iloc[i],
            "facilitiesServices": data['facilitiesServices'].iloc[i]    
        }

In [130]:
# Calculate TF-IDF for the query

def calculate_query_tfidf(preprocess_query, dict_word_doc_id, N):
    tf_query = {}
    # Calculate Term Frequency (TF) for each word in the query
    for word in preprocess_query:
        tf_query[word] = tf_query.get(word, 0) + 1

    number_word = len(preprocess_query)

    # Calculate TF-IDF scores for the query
    tfidf_query = {}
    for word, tf in tf_query.items():
        
        # Only include words that exist in our document collection
        if word in dict_word_doc_id:
            normalized_tf = tf / number_word  # Normalize TF by query length
            idf = math.log(N / len(dict_word_doc_id[word]))  # Calculate IDF
            tfidf_query[word] = normalized_tf * idf

    return tfidf_query

# Cosine Similarity calculation
def cosine_similarity(vector1, vector2):
    # Calculate dot product
    dot_product = sum(vector1[word] * vector2.get(word, 0) for word in vector1)

    # Calculate Euclidean norms
    norm1 = math.sqrt(sum(value ** 2 for value in vector1.values()))
    norm2 = math.sqrt(sum(value ** 2 for value in vector2.values()))

    # Ensure norms are non-zero to prevent division by zero
    return dot_product / (norm1 * norm2) if norm1 and norm2 else 0

# Ranking function to get top k results
def ranking_function(query, tf_idf_data, k, dict_word_doc_id, N, restaurant_info_dict):
    
    # Calculate TF-IDF for the query
    tfidf_query = calculate_query_tfidf(query, dict_word_doc_id, N)

    # Calculate cosine similarity for each document
    cos_sim = []
    for document_id, tfidf_vector in tf_idf_data.items():
        score = cosine_similarity(tfidf_query, tfidf_vector)
        if score > 0:  # Only consider documents with a positive similarity score
            cos_sim.append((document_id, score))

    # Sort results by similarity score in descending order and take top k
    cos_sim_sorted = sorted(cos_sim, key=lambda x: x[1], reverse=True)[:k]

    # Create the final results with details
    final_results = []
    for document_id, score in cos_sim_sorted:
        restaurant = restaurant_info_dict[document_id]
        final_results.append({
            "restaurantName": restaurant['restaurantName'],
            "address": restaurant['address'],
            "description": restaurant['description'],
            "website": restaurant['website'],
            "Similarity score": round(score, 4)
        })

    return pd.DataFrame(final_results)


In [143]:
# Prepare the query
query = input("Please enter a short description of the type of restaurant you're looking for:\n" "For example: 'modern seasonal cuisine with a terrace' or 'romantic Italian restaurant'.\n\n"
              "Your description: ")
preprocess_query=text_cleaning(query)
k=5

# Execute Ranking function
results=ranking_function(preprocess_query,tf_idf,k,dict_word_doc_id,len(data),restaurant_info_dict)

# Show the result 
results

Please enter a short description of the type of restaurant you're looking for:
For example: 'modern seasonal cuisine with a terrace' or 'romantic Italian restaurant'.

Your description:  modern seasonal cuisine


,restaurantName,address,description,website,Similarity score
0,Matteo Ristorante,piazza Duomo 6,"The soft-toned decor is modern and intimate, while the cuisine also has a contemporary feel. Dishes include meat and fish options, plus numerous types of risottos, many of which feature distinctly modern ingredients. Coffee service and aperitifs in the near Laboratory, at n 10.",https://matteocaffeecucina.it/,0.4056
1,Il Cavallo Scosso,via al Duca 23/d,"A young and modern restaurant situated in a residential area about 2km from the centre, where the owner-chef offers three tasting menus:Territorio e Tradizione,Il Cavallo Scossoand the more creativeFuori Gara.",https://www.ilcavalloscosso.it/,0.0744
2,Casa Perbellini 12 Apostoli,vicolo Corticella San Marco 3,"Giancarlo Perbellini returns to his origins in this historic restaurant in his native city, where every chef from Verona (and possibly beyond) would like to work. The updated decor gives the restaurant even more appeal, as does the culinary style and philosophy showcased on three tasting menus: “Io e Silva”, which includes decidedly imaginative dishes (such as cooked and raw shellfish with a dash of soya and peppers) and is dedicated to his wife; “Io e Giorgio”, dedicated to his mentor and former restaurant owner, which features more classic recipes; and, last but not least, the completely vegetarian “L'Essenza”. Special mention must be made of the wine list, which includes an extensive selection of French labels, of which both the sommelier and Giancarlo are huge fans. As a final attraction, make sure you find time to visit the Roman ruins in the basement. We also highly recommend booking the chef's table where you can dine as a couple yet at the same time enjoy the company of the skilled chefs whom you’ll observe working together like the well-practised members of an orchestra – an extraordinary sight!",http://www.casaperbellini.com,0.0727
3,Trattoria Pennestri,via Giovanni Da Empoli 5,"An authentic neighbourhood trattoria with just one difference – it is so good that its fame has travelled far beyond the streets of Ostiense in which it is situated and so it is often crowded (as a result, booking is highly recommended). The atmosphere is simple, informal and attractive, while the cuisine focuses on Roman classics such as pasta with carbonara, cacio e pepe, amatriciana and gricia sauces, alongside a few more creative dishes.",https://trattoriapennestri.it/,0.0686
4,I Due Buoi,via Vittorio Veneto 23,"This restaurant offers three tasting menus: Stile Libero, Tradizione e Territorio, and Quinto Quarto , the latter featuring various types of offal which the chef transforms into creative dishes, plus an à la carte that includes local specialities and a good choice of bread. Top-quality wines with a focus on Asti and its surrounding area, as well as on the Langhe region.",https://www.iduebuoi.it,0.0676


<h1> <b> 3.0.0 Define a New Score! </b> <img src= "img point 3.PNG" width=150 style="vertical-align: middle"> </h1>

## New Scoring function:

### Define a scoring function that takes into account various attributes:

- #### `Description Match`: Give weight based on the query similarity to the description (using TF-IDF scores).
- #### `Cuisine Match`: Increase the score for matching cuisine types.
- #### `Facilities and Services`: Give more points for matching facilities/services (e.g., “Terrace,” “Air conditioning”).
- #### `Price Range`: Higher scores could be given to more affordable options based on the user’s choice.

In [132]:
# Scoring function that incorporates description, cuisine, services, and price range
def custom_scoring(query,tfidf_query, tfidf_vector,restaurant_info_dict,document_id):
    cuisine_score=0
    servicies_score=0
    restaurant=restaurant_info_dict[document_id]
    # 1. Description Match (Cosine Similarity)
    description_score = cosine_similarity(tfidf_query, tfidf_vector)

    # 2. Cuisine Match (Boost score for matching cuisine type)
    if any(cuisine in query for cuisine in restaurant['cuisineType'].lower().split(',')):
        cuisine_score += 1  # Add a constant boost for matching cuisine type

    # 3. Facilities and Services Match (Add points for matching facilities/services)
    if restaurant['facilitiesServices']:
        if any(facility.lower() in query for facility in restaurant['facilitiesServices']):
            servicies_score += 1  # Add a boost for matching services

    # 4. Price Range (Match based on user’s budget, if provided)
    if restaurant['priceRange']=="Economic" or restaurant['priceRange']=="Affordable": 
            price_score = 1 
    else: 
        price_score=0.5
        
    return description_score*0.4 + cuisine_score*0.3 + servicies_score*0.2 + price_score*0.1

# Main Ranking Function that uses Heap for Top-k

def ranking_function_with_custom_score(query, tf_idf_data, k, dict_word_doc_id, N ,restaurant_info_dict):
    
    # Preprocess query for TF-IDF calculation
    preprocess_query = query.lower().split()
    tfidf_query = calculate_query_tfidf(preprocess_query, dict_word_doc_id, N)

    # Use a heap to maintain the top-k restaurants based on custom score
    heap = []

    for restaurant_id, tfidf_vector in tf_idf_data.items():
        
        score = custom_scoring(query,tfidf_query, tfidf_vector,restaurant_info_dict,restaurant_id)

        # Maintain the top-k heap (min-heap)
        if len(heap) < k:
            heapq.heappush(heap, (score, restaurant_id))
        else:
            heapq.heappushpop(heap, (score, restaurant_id))

    # Extract the top-k results
    top_k_results = []
    for score, restaurant_id in heap:
        restaurant = restaurant_info_dict[restaurant_id]
        top_k_results.append({
            "restaurantName": restaurant['restaurantName'],
            "address": restaurant['address'],
            "description": restaurant['description'],
            "website": restaurant['website'],
            "Cuisine Type": restaurant.get('cuisineType', ''),
            "Facilities and Services": restaurant.get('facilitiesServices', ''),
            "Price Range": restaurant.get('priceRange', ''),
            "Custom Score": round(score, 4)
        })
    top_k_results = sorted(top_k_results, key=lambda x: x['Custom Score'], reverse=True)
    return pd.DataFrame(top_k_results)

In [145]:
# Example usage and results:

query = input("Please enter a short description of the type of restaurant you're looking for:\n" "For example: 'modern seasonal cuisine with a terrace' or 'romantic Italian restaurant'.\n\n"
              "Your description: ")
k = 5
results = ranking_function_with_custom_score(query, tf_idf, k, dict_word_doc_id, len(data), restaurant_info_dict)
results

Please enter a short description of the type of restaurant you're looking for:
For example: 'modern seasonal cuisine with a terrace' or 'romantic Italian restaurant'.

Your description:  modern seasonal cuisine


,restaurantName,address,description,website,Cuisine Type,Facilities and Services,Price Range,Custom Score
0,Piccolo Lord,corso San Maurizio 69 bis/g,"Professional service in a welcoming, modern restaurant run by a young couple. He works in the kitchen while she (having also worked as a chef in the past) runs the front of house. Delicious Mediterranean cuisine with a seasonal focus.",https://www.ristorantepiccololord.it/,"Mediterranean Cuisine, Seasonal Cuisine",[Air conditioning],Affordable,0.4487
1,Altriménti,via Monte Bianco 2/a,"An informal and contemporary restaurant which is elegant and yet retains a bistro feel. The cuisine is modern with meat and fish dishes featured on the menu, as well as a good number of vegetarian options. Good wine list.",https://altrimenti.eu/,"Modern Cuisine, Seasonal Cuisine",[],Affordable,0.4450
2,Kirchsteiger,via prevosto Wieser 5,This restaurant has long been one of the most interesting in the area. It has a pleasing combination of traditional and modern cuisine created by the expert chef. The dining rooms have the same blend of old and new with an elegant use of local materials in the decor. Ask for one of the newer guestrooms.,https://www.kirchsteiger.com/it/gastronomia/ristorante,"Classic Cuisine, Seasonal Cuisine","[Car park, Garden or park, Great view, Interesting wine list, Terrace]",Affordable,0.4360
3,Mima,via Madonnelle 9,"You’ll be won over by the seasonal Mediterranean cuisine created by the young yet experienced chef at this restaurant. Accommodation is also available in modern guestrooms, plus there’s an enchanting roof garden in which to sip an aperitif while the sun goes down.",http://www.domo20.com/restaurant,"Seasonal Cuisine, Mediterranean Cuisine","[Air conditioning, Great view, Terrace]",Affordable,0.4353
4,Trattoria 'petito,via Corridoni 14,"Located on the outskirts of Forlì’s charming historic centre, this simple, modern yet attractive restaurant serves eclectic and varied cuisine. More traditional fare on offer includes an excellent selection of hams and grilled meats, as well as delicious tagliatelle al ragù, while the menu also features a few more creative options including a few fish dishes.",https://www.trattoriapetito.it/,"Cuisine from Romagna, Seasonal Cuisine","[Air conditioning, Car park, Terrace, Wheelchair access]",Affordable,0.4324


<h1> <b> Cosine Similarity </b> <img src="vs-removebg-preview.png" width=80 style="vertical-align: middle"> <b> Custom Score </b> </h1>


The personalized scoring function provides superior results by going beyond simple keyword matching based solely on cosine similarity values based on the **TF-IDF** vectors of the query and each document. This new function takes into account more factors related to user preferences, providing more relevant matches while improving user satisfaction. 

---

## Comparison of two methods

The search results differ significantly between the methods:

- Using only **cosine similarity**, *"Matteo Ristorante"* scored **0.4**, while the remaining **k restaurants** scored between **0** and **0.1**, indicating that considering only this parameter is not a good criterion.

- In contrast, the **personalized scoring method**, which considers more factors related to user preferences, provided higher matching scores in all k restaurants, demonstrating that considering more related factors can help in building a better search engine.

